# Frequent pattern mining - Exercise 2

## Exercise 2-1 - Combinatorial Explosion  

**A: A database contains transactions over the following items: “apples”, “bananas”, and “cherries”. How many different combinations of these items can exist (i.e., how many different transactions could possibly occur in the database)?**  

This can be solved with the formula 2^n which is 2^3 in this case, which yields 8.  

In [1]:
def get_powerset(s, verbose = False):
    x = len(s)

    return_set = []
    for i in range(1 << x):
        sub_set = [s[j] for j in range(x) if (i & (1 << j))]
        return_set.append(sub_set)
        if verbose:
            print(sub_set)
    return return_set

def get_powerset_length(x):
    """ 
    Formula is 2**n where n is the length of the set
    """
    return 2**len(x)


my_set = ["apple", "cherry", "banana"]
power_set = get_powerset(my_set, verbose=True)
print(get_powerset_length(my_set))

[]
['apple']
['cherry']
['apple', 'cherry']
['banana']
['apple', 'banana']
['cherry', 'banana']
['apple', 'cherry', 'banana']
8


**B: The database now also contains the items “dates”, “eggplants”, “figs”, and “guavas”. How many possible transactions do we have now?**   

Again now the formel is 2^6, which yields 64

In [2]:
my_set = ["apple", "cherry", "banana", "dates", "eggplants", "figs"]
#power_set = get_powerset(my_set)
print(get_powerset_length(my_set))

64


**C: How many combinations (possible different transactions) do we have with n items?**  

Again it is just 2^n

**D: How many transactions with exactly two items (i.e., 2-itemsets) can we have when the database contains
3 items? When it contains 5 items? How many k-itemsets do we have when the database contains n
items?**  

This is calculated with the binomial coefficent formula, which is n! / ( k! * (n - k)! )

In [3]:
def get_k_itemsets(power_set, k, verbose = False):
    return_set = []
    for sub_set in power_set:
        if len(sub_set) == k:
            return_set.append(sub_set)
            if verbose:
                print(sub_set)
    return return_set


from math import factorial
def get_len_of_k_itemsets(n, k):
    """
    Calcualtes how many k itemsets of length k there will be in the superset
    n can either be list of items or an integer, which indicates the length of the itemset
    """
    if isinstance(n, list):
        n = len(n)
    else:
        n = n

    if k > n:
        return 0
    else:
        return factorial(n) / (factorial(k) * factorial(n - k)) 

my_set = ["apple", "cherry", "banana", "dates", "eggplants"]
print(get_len_of_k_itemsets(my_set, 2))


my_set = ["apple", "cherry", "banana"]
print(get_len_of_k_itemsets(my_set, 2))




10.0
3.0


## 2-2: Itemsets and Association Rules

Given a set of transactions T according to the following table:  

In [4]:
import pandas as pd
df = pd.DataFrame(
    {"items_in_basket": [["milk", "beer", "diapers"],
                         ["bread", "butter", "milk"],
                         ["milk", "diapers", "cookies"],
                         ["bread", "butter", "cookies"],
                         ["beer", "cookies", "diapers"],
                         ["milk", "diapers", "bread", "butter"],
                         ["bread", "butter", "diapers"],
                         ["beer", "diapers"],
                         ["milk", "diapers", "bread", "butter"],
                         ["beer", "cookies"]]}
)

df

,items_in_basket
0,"[milk, beer, diapers]"
1,"[bread, butter, milk]"
2,"[milk, diapers, cookies]"
3,"[bread, butter, cookies]"
4,"[beer, cookies, diapers]"
5,"[milk, diapers, bread, butter]"
6,"[bread, butter, diapers]"
7,"[beer, diapers]"
8,"[milk, diapers, bread, butter]"
9,"[beer, cookies]"


**A: What are the support and the confidence of {milk} -> {diapers}**  

The support is retrieved by counting how often all the components (ie. milk and diapers) occur in the same transaction. In this case 4 times.  

The confidence is retrieved by by dividing the support of the association rule with the support of the antecetent (left side). which in this case is 0.8

In [5]:
transactions = list(df["items_in_basket"])

def get_support(transactions: list[str], antecedent: list[str], consequent: list[str] = [], fraction: bool = False):
    """
    Takes a list of transactions, a antecedent (left of of the rule) and a consequent (right part of the rule)
    returns the support of the rule.
    The support is retrieved by counting how often all the components (ie. milk and diapers) occur in the same transaction
    If you only wish to get the support of an itemset, just leave consequent blank.
    """
    association_rule = set(consequent + antecedent)
    support = 0
    for transaction in transactions:
        transaction = set(transaction)
        if set.intersection(association_rule, transaction) == association_rule:
            support += 1
    if fraction:
        support = support / len(transactions)
    return support



def get_confidence(transactions: list[str], antecedent: list[str], consequent: list[str] = []):
    """
    Takes a list of transactions, a antecedent (left of of the rule) and a consequent (right part of the rule)
    returns the confidence of the rule
    
    The formula is given by dividing the support of the rule with the support of the antecedent
    """
    return get_support(transactions, antecedent, consequent) / get_support(transactions, antecedent)


support = get_support(transactions, antecedent = ["milk"], consequent = ["diapers"])
confidence = get_confidence(transactions, antecedent = ["milk"], consequent = ["diapers"])

print(f"Support: {support}")
print(f"Confidence: {confidence}")

Support: 4
Confidence: 0.8


**B: What are the support and confidewnce of {diapers} -> {milk}?**  



In [6]:
support = get_support(transactions, antecedent = ["diapers"], consequent = ["milk"])
confidence = get_confidence(transactions, antecedent = ["diapers"], consequent = ["milk"])

print(f"Support: {support}")
print(f"Confidence: {confidence}")

Support: 4
Confidence: 0.5714285714285714


**C: What is the maximum number of size-3 itemsets that can be derived from this data set?**  

This is done by finding the transactions with the most number of items and calculating how many subsets it can contain, (the binomial coefficent). Here we will use the function from previously


In [7]:
def find_max_list(list: list[str]):
    """
    This finds the length of the longest list in a list of list.
    It does NOT work with multiple nested lists.
    """
    return max([len(i) for i in list])

max_itemsets = get_len_of_k_itemsets(n = find_max_list(transactions),
                                     k = 3)
print(f"Maximum number of 3-itemsets: {max_itemsets}")

Maximum number of 3-itemsets: 4.0


**D: What is the maximum number of association rules that can be extracted from this dataset (including rules that have zero support)?**  
 
 The formula is from the book data mining second edition Tan. ( (3^r) - (2^(r + 1)) ) + 1

In [8]:
def get_unique_items(transactions):
    unique_items = []
    for transaction in transactions:
        for item in transaction:
            if not (item in unique_items):
                unique_items.append(item)
    return unique_items    

def get_max_association_rules(transactions: list[list[str]] | int):
    """
    Returns the maximum amount of association rules, that are possible to generate, without empty sets
    it uses the fomula ( (3**r) - (2**(r + 1)) ) + 1, where r is the amount of unique items in the database
    it does not consider empty sets.
    """
    if isinstance(transactions, list):
        unique_items = get_unique_items(transactions)
        r = len(unique_items)
    else:
        r = transactions
    return ( (3**r) - (2**(r + 1)) ) + 1

print(f"Max association rules are: {get_max_association_rules(transactions)}")

Max association rules are: 602


**E: What is the maximum size of the frequent itemsets that can be extracted (assuming support > 0 )?**

The maximum support is 4, since the itemset {milk, diapers, bread, butter} occurs once and has a support of 2


**F: Find an itemset (of size 2 or larger) that has the largest support**  


In [151]:
from itertools import combinations
from itertools import permutations
def get_itemset_with_highest_support(transactions: list[list[str]], size: int | None = None):
    """
    FIXME
    """
    unique_items = get_unique_items(transactions)
    if size:
        combs = [list(comb) for comb in combinations(unique_items, size)]
    else:
        combs = [list(comb) for comb in combinations(unique_items)]

    support = {}
    for comb in combs:
        support[str(comb)] = get_support(transactions, comb)

    return dict(sorted(support.items(), key=lambda item: item[1], reverse=True))
    #power_set = get_powerset(get_unique_items(transactions))
    #print(power_set)

get_itemset_with_highest_support(transactions, size = 2)

{"['G', 'B']": 8,
 "['H', 'E']": 7,
 "['H', 'B']": 7,
 "['E', 'B']": 7,
 "['B', 'A']": 6,
 "['H', 'G']": 5,
 "['H', 'A']": 5,
 "['E', 'G']": 5,
 "['E', 'A']": 5,
 "['E', 'C']": 5,
 "['E', 'F']": 5,
 "['H', 'C']": 4,
 "['H', 'F']": 4,
 "['G', 'A']": 4,
 "['G', 'D']": 4,
 "['B', 'F']": 4,
 "['B', 'D']": 4,
 "['C', 'F']": 4,
 "['G', 'F']": 3,
 "['B', 'C']": 3,
 "['A', 'C']": 3,
 "['H', 'D']": 2,
 "['H', 'K']": 2,
 "['E', 'K']": 2,
 "['G', 'C']": 2,
 "['B', 'K']": 2,
 "['A', 'F']": 2,
 "['A', 'D']": 2,
 "['F', 'D']": 2,
 "['H', 'L']": 1,
 "['H', 'I']": 1,
 "['E', 'D']": 1,
 "['E', 'L']": 1,
 "['E', 'I']": 1,
 "['G', 'L']": 1,
 "['G', 'K']": 1,
 "['G', 'I']": 1,
 "['B', 'L']": 1,
 "['B', 'I']": 1,
 "['A', 'K']": 1,
 "['C', 'D']": 1,
 "['C', 'L']": 1,
 "['F', 'L']": 1,
 "['F', 'K']": 1,
 "['F', 'I']": 1,
 "['D', 'L']": 1,
 "['K', 'I']": 1,
 "['A', 'L']": 0,
 "['A', 'I']": 0,
 "['C', 'K']": 0,
 "['C', 'I']": 0,
 "['D', 'K']": 0,
 "['D', 'I']": 0,
 "['L', 'K']": 0,
 "['L', 'I']": 0}

**G: Find a pair of items a and b such that the rule {a} -> {b} and {b} -> {a} have the same confidence**  

How to do this is simple: FInd items, who has the same support, and then test them together until you find some with same confidence. But remember, confidence is not always symetrical, it is possible to have different values depending on which is antecedent and consequent

In [103]:
print(f"Milk and bread conf: {get_confidence(transactions, ['milk'], ['bread'])} ")

print(f'Bread and milk conf: {get_confidence(transactions, ["bread"], ["milk"])} ')


Milk and bread conf: 0.6 
Bread and milk conf: 0.6 


## 2-3 Apriori candidate generation  
Given the frequent 3-itemsets:  

    {1, 2, 3}, {1, 2, 4}, {1, 2, 5}, {1, 3, 4}, {1, 3, 5}, {2, 3, 4}, {2, 3, 5}, {3, 4, 5}  
    
List all candidate 4-itemsets following the Apriori joining and pruning procedure

In [175]:
def _apriori_join(items):
    
    candidates = []
    for i, itemset_outer in enumerate(items):
        itemset_outer = sorted(list(itemset_outer))
        
        for j, itemset_inner in enumerate(items):
            itemset_inner = sorted(list(itemset_inner))
            if j != i and itemset_inner[0: - 1] == itemset_outer[0: - 1]:
                candidate = set(itemset_inner + itemset_outer)
                if candidate not in candidates:
                    candidates.append(set(itemset_inner + itemset_outer))
    return candidates                    


def _apriori_prune(candidates, items, k):
    
    pruned_candidates = []
    for i, candidate in enumerate(candidates):
        subsets = [set(sorted(subset)) for subset in combinations(candidate, k)]
        for subset in subsets:
            valid_subset = False
            
            for item in items:
                if item.issubset(candidate):
                    valid_subset = True
                    break

        
        if valid_subset:
            pruned_candidates.append(candidate)
    return pruned_candidates


def apriori_generate_candidates(items: list[set[str | int]]):
    """
    Public function for generating apriori candidates
    """

    n = len(items[0])
    candidates = _apriori_join(items)
    pruned_candidates = _apriori_prune(candidates, items, n)
    return pruned_candidates


def aprio_algo(itemset, support):
    
    
    for i in range(len(itemset)):
        itemset[i] = set(sorted(itemset[i]))
    unique_items = get_unique_items(itemset)
    
    
    s_1 = []
    for item in unique_items:
        if get_support(itemset, [item]) > support:
            item =item
            s_1.append(item)
    
    print(s_1)        
    candidates = [set(subset) for subset in combinations(s_1, 2)]
    
    pruned_candidates = []
    for candidate in candidates:
        
        count = 0
        for item in itemset:
            if candidate.issubset(item):
                
                count += 1 
        
        if count > support:
            pruned_candidates.append(candidate)
    
    
    candidates = pruned_candidates
    print(candidates)
    while len(candidates) > 0:
        candidates = apriori_generate_candidates(candidates)
        pruned_candidates = []
        for candidate in candidates:
            count = 0
            for item in itemset:
                if candidate.issubset(item):
                    count += 1 
            if count > support:
                pruned_candidates.append(candidate)
                
                
        print(candidates)
        candidates = pruned_candidates
        print(candidates)

        
        
        
        
    
print(apriori_generate_candidates([{1, 2, 3}, {1, 2, 4}, {1, 2, 5}, {1, 3, 4}, {1, 3, 5}, {2, 3, 4}, {2, 3, 5}, {3, 4, 5}  ]))


[{1, 2, 3, 4}, {1, 2, 3, 5}, {1, 2, 4, 5}, {1, 3, 4, 5}, {2, 3, 4, 5}]


![Title](imgs/Screenshot_4.png)

In [176]:
transactions = [{"A", "B", "E"},
                {"B", "D"},
                {"C", "D", "F"},
                {"A", "B", "D"},
                {"A", "C", "E"},
                {"B", "C", "E", "F"},
                {"A", "C", "E"},
                {"A", "B","C", "E"},
                {"A", "B", "C", "D", "F"},
                {"B", "C", "D", "E"},
                ]

aprio_algo(transactions, support = 2)



['A', 'E', 'B', 'D', 'F', 'C']
[{'A', 'E'}, {'A', 'B'}, {'A', 'C'}, {'B', 'E'}, {'C', 'E'}, {'D', 'B'}, {'C', 'B'}, {'D', 'C'}, {'F', 'C'}]
[{'A', 'E', 'B'}, {'A', 'C', 'E'}, {'A', 'C', 'B'}, {'D', 'E', 'B'}, {'C', 'E', 'B'}, {'D', 'C', 'E'}, {'F', 'C', 'E'}, {'D', 'C', 'B'}, {'D', 'F', 'C'}]
[{'A', 'C', 'E'}, {'C', 'E', 'B'}]
[]
[]


## FULL apriori

In [164]:
def _apriori_join(items):
    
    candidates = []
    for i, itemset_outer in enumerate(items):
        itemset_outer = sorted(list(itemset_outer))
        
        for j, itemset_inner in enumerate(items):
            itemset_inner = sorted(list(itemset_inner))
            if j != i and itemset_inner[0: - 1] == itemset_outer[0: - 1]:
                candidate = set(itemset_inner + itemset_outer)
                if candidate not in candidates:
                    candidates.append(set(itemset_inner + itemset_outer))
    return candidates                    


def _apriori_prune(candidates, items, k):
    
    pruned_candidates = []
    for i, candidate in enumerate(candidates):
        subsets = [set(sorted(subset)) for subset in combinations(candidate, k)]
        for subset in subsets:
            valid_subset = False
            
            for item in items:
                if item.issubset(candidate):
                    valid_subset = True
                    break

        
        if valid_subset:
            pruned_candidates.append(candidate)
    return pruned_candidates


def apriori_generate_candidates(items: list[set[str | int]]):
    """
    Public function for generating apriori candidates
    """

    n = len(items[0])
    candidates = _apriori_join(items)
    pruned_candidates = _apriori_prune(candidates, items, n)
    return pruned_candidates


def aprio_algo(itemset, support):
    
    
    for i in range(len(itemset)):
        itemset[i] = set(sorted(itemset[i]))
    unique_items = get_unique_items(itemset)
    
    
    s_1 = []
    for item in unique_items:
        if get_support(itemset, [item]) > support:
            item =item
            s_1.append(item)
            
    candidates = [set(subset) for subset in combinations(s_1, 2)]
    
    
    pruned_candidates = []
    for candidate in candidates:
        
        count = 0
        for item in itemset:
            if candidate.issubset(item):
                
                count += 1 
        
        if count > support:
            pruned_candidates.append(candidate)
    
    
    candidates = pruned_candidates
    while len(candidates) > 0:
        prev_candidates = candidates
        candidates = apriori_generate_candidates(candidates)
        pruned_candidates = []
        for candidate in candidates:
            count = 0
            for item in itemset:
                if candidate.issubset(item):
                    count += 1 
            if count > support:
                pruned_candidates.append(candidate)
                
                
        
        candidates = pruned_candidates
    return prev_candidates




transactions = [{"B", "E", "G", "H"},
                {"A", "B", "C", "E", "G", "H"},
                {"A", "B", "C", "E", "F", "H"},
                {"B", "C", "D", "E", "F", "G", "H", "L"},
                {"A", "B", "E", "K", "H"},
                {"B", "E", "F", "G", "H", "I", "K"},
                {"A", "B", "D", "G", "H"},
                {"A", "B", "D", "G"},
                {"B", "D", "F", "G"},
                {"C", "E", "F"},
                {"A", "C", "E", "F", "H"},
                {"A", "B", "E", "G"}
                ]

freq_items = aprio_algo(transactions, support = 3)


for freq_item in freq_items:
    freq_item = list(freq_item)
    
    for i in range(len(freq_item)):
        i = i + 1
        subsets = [set(sorted(subset)) for subset in combinations(freq_item, i)]
        print(subsets)
        # Here we can see all those that are missing, are the ones that are the consequent
    
        
            

[{'H'}, {'E'}, {'G'}, {'B'}]
[{'H', 'E'}, {'H', 'G'}, {'H', 'B'}, {'G', 'E'}, {'E', 'B'}, {'G', 'B'}]
[{'H', 'G', 'E'}, {'H', 'E', 'B'}, {'H', 'G', 'B'}, {'E', 'G', 'B'}]
[{'H', 'E', 'G', 'B'}]


# Exercise 3  
Support = 4  

![title](imgs/Screenshot_6.png)

Support = 2  

![title](imgs/Screenshot_7.png)

# Exam sets

![title](imgs/Screenshot_1.png)

In [167]:
l_3 = [{"A", "B", "H"}, {"A", "C", "F"}, {"A", "C", "G"}, {"A", "E", "H"}, {"A", "F", "G"}, {"A", "F", "H"}, {"A", "G", "H"}, {"A", "H", "I"},
       {"B", "E", "H"}, {"B", "E", "I"}, {"B", "F", "H"}, {"C", "F", "G"}, {"D", "E", "H"}, {"E", "H", "I"}, {"F", "H", "I"}]

_apriori_join(l_3)

[{'A', 'C', 'F', 'G'}, {'A', 'F', 'G', 'H'}, {'B', 'E', 'H', 'I'}]

![title](imgs/Screenshot_2.png)  

Just like the anti-monotone property of support, confidence of rules generated from the same itemset also follows an anti-monotone property. It is anti-monotone with respect to the number of elements in consequent.
This means that confidence of (A,B,C→ D) ≥ (B,C → A,D) ≥ (C → A,B,D). To remind, confidence for {X → Y} = support of {X,Y}/support of {X} 

In taken from the towards data science post. This means that if there are more values on the right hand side, then it will definitely be below the previous one.  


![title](imgs/Screenshot_8.png)